In [195]:
import numpy as np
import random

Read In Data

In [196]:
Data_File = open('Admissions.txt','r') #open the file contents
Data_String = Data_File.read() # read the contents into a string
full_data = Data_String.splitlines()# storing each datapoint into a list
random.shuffle(full_data) # for random sampling of training, validation and testing data

Analysing and processing the Data
First we will remove our first column

In [202]:
new_full_data = []
for i in range(len(full_data)): # removing the first column from the full data
    list = full_data[i].split(",")
    list.pop(0)
    str = ""
    for i in range(len(list)):
        str = str + list[i] 
        if(i < (len(list)-1)):
            str = str + ","
    list = []
    new_full_data.append(str)

data_adjust = [] # to store the new adjusted data
for i in range(len(new_full_data)): # normalising our targets from being continuos to being either 0 or 1 with 0.5 as the threshold
    list = new_full_data[i].split(",")
    Chance = list[-1] 
    if(float(Chance) > 0.5):  # the deciding value/ threshold is 0.5
        Chance = "1"
    else:
        Chance = "0"
    list[-1] = Chance
    str = ""
    for i in range(len(list)):
        str = str + list[i] 
        if(i < (len(list)-1)):
            str = str + ","
    list = []
    data_adjust.append(str)  # new adusted data is stored in data_adjust


    

DIVIDING THE DATA

240 data points- Training data.

80 data points- Validation data.

80 data points- Test data.

In [203]:
training_data =  data_adjust[:240]
validation_data = data_adjust[240:320]
test_data = data_adjust[320:400]

PREDICTION ALGORITHM 1 : LINEAR REGRESSION